In [5]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import os.path

In [6]:
import xml.etree.ElementTree as etree

In [7]:
def build_df(df):
    df = pd.DataFrame()
    for i in range(4,137028):
        dict_new = root[i].attrib
        df = df.append(dict_new, ignore_index=True)
    return df    

In [8]:
def wrangle_xml():
    
    if os.path.isfile('./xml.csv'):
        df = read_csv('xml.csv')
    
    else:
   
        tree = etree.parse("export.xml")
    
        root = tree.getroot()
    
        df = build_df(df)
    
        df.to_csv('xml.csv')
    
    return df

In [ ]:
def validate_data_sets():
    df = pd.DataFrame(root[4].attrib, index=[0])
    
    print(root[5].attrib['type'])
    
    ## check to see if attribute is step counter
    i = 4
    while i < 67920:
        if root[i].attrib['type'] == 'HKQuantityTypeIdentifierStepCount':
        else:
            print(i)
            break
            i += 1
            
    root[67919].attrib
    
    i = 67919
    while i < 1_000_000:
        if root[i].attrib['type'] == 'HKQuantityTypeIdentifierDistanceWalkingRunning':
        else:
            print(f'{i} is false and break')
            break
        i += 1
        
    root[135827].attrib
    
    i = 135827
    while i < 1_000_000:
        if root[i].attrib['type'] == 'HKQuantityTypeIdentifierFlightsClimbed':
        else:
            print(f'{i} is false and break')
            break
        i += 1
        
    root[137028].attrib
    
    i = 
    type_list = []
    while i < 1_000_000:
        if root[i].attrib['type'] in type_list:

        else:
            type_list.append(root[i].attrib['type'])
        i +=1
    
    type_list
    
    return type_list

In [9]:
import os.path

In [10]:
os.path.isfile('./xml.csv')

True

In [15]:
from wrangle_xml import wrangle_xml

In [17]:
df = wrangle_xml()

UnboundLocalError: local variable 'df' referenced before assignment

In [ ]:
df.to_csv('xml.csv')

In [ ]:
df['endDate'] = pd.to_datetime(df.endDate)
df['startDate'] = pd.to_datetime(df.startDate)
df['creationDate'] = pd.to_datetime(df.creationDate)

In [ ]:
df = df.set_index('startDate')

In [ ]:
df.value.resample('D').sum()

In [ ]:
df.type.value_counts()

In [ ]:
df = pd.read_csv("xml.csv")

In [ ]:
df.type.value_counts()

In [ ]:
df_steps = df[df.type == 'HKQuantityTypeIdentifierStepCount']

In [ ]:
df_steps

In [ ]:
df_steps['endDate'] = pd.to_datetime(df_steps.endDate)
df_steps['startDate'] = pd.to_datetime(df_steps.startDate)
df_steps['creationDate'] = pd.to_datetime(df_steps.creationDate)

In [ ]:
df_steps = df_steps[['creationDate', 'endDate', 'startDate', 'value']]

In [ ]:
df_start = pd.DataFrame(df_steps.set_index('startDate').resample('d').value.sum())

In [ ]:
df_start